In [2]:
# Display the current working directory
getwd();
# If necessary, change the path below to the directory where the data files are stored.
# "." means current directory. On Windows use a forward slash / instead of the usual \.
workingDir = "/home/jovyan/work/research/topics/datasets/single-cell";
setwd(workingDir);
# Load WGCNA package
library(WGCNA)
allowWGCNAThreads()
# The following setting is important, do not omit.
options(stringsAsFactors = FALSE);

[1] "/home/jovyan/work/research/topics/datasets/single-cell"

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




Allowing multi-threading with up to 12 threads.


In [3]:
add_legend <- function(...) {
  opar <- par(fig=c(0, 1, 0, 1), oma=c(0, 0, 0, 0), 
    mar=c(0, 0, 0, 0), new=TRUE)
  on.exit(par(opar))
  plot(0, 0, type='n', bty='n', xaxt='n', yaxt='n')
  legend(...)
}

In [4]:
datExpr <- t(read.csv("mainTable.csv", check.names=FALSE, row.names = ""))
genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples
allTraits <- read.csv("files.dat")

In [5]:
log2_1 <- function(x){
    log2(x+1)
}
datExpr <- as.data.frame(apply(datExpr,2, log2_1))
dimnames(datExpr)[[1]] <- samples

In [6]:
datExpr[1:10,1:5]

,A2M,AAMP,AASDHPPT,AB019441.29,ABAT
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AdultCerebellum_1.ACGTTGCCAGACGATCTT,0,0,0.0000000,0,0
AdultCerebellum_1.CGAGTACGCACCGAGATC,0,0,0.0000000,0,0
AdultCerebellum_1.GTATACGAATTAAATAAA,0,0,0.0000000,0,0
AdultCerebellum_1.ATCTCTGCAGGATGCGGA,0,0,0.0000000,0,0
AdultCerebellum_1.CGAGTACTGTGTGCGAAT,0,0,0.7597074,0,0
AdultCerebellum_1.ACGAGCTAGAGACTCCAT,0,0,0.7597074,0,0
AdultCerebellum_1.ACGAGCCTTCTGGCCCTC,0,0,0.7597074,0,0
AdultCerebellum_1.ATCAACAAAGTTTAAGGG,0,0,0.0000000,0,0
AdultCerebellum_1.CCTTTCGACACTTTCATA,0,0,0.0000000,0,0


In [7]:
head(allTraits)

,Cell_id,Sample,Batch,Cluster_id,Ages,Development_stage,Method,Gender,Source,Biomaterial,Name,ident,Celltype
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,AdultCerebellum_1.ACGTTGCCAGACGATCTT,AdultCerebellum,AdultCerebellum1,AdultCerebellum1_1,55Y,Adult,Microwell-seq,Female,HCL,AdultCerebellum,AdultCerebellum_55Y,1,Oligodendrocyte
2,AdultCerebellum_1.CGAGTACGCACCGAGATC,AdultCerebellum,AdultCerebellum1,AdultCerebellum1_1,55Y,Adult,Microwell-seq,Female,HCL,AdultCerebellum,AdultCerebellum_55Y,1,Oligodendrocyte
3,AdultCerebellum_1.GTATACGAATTAAATAAA,AdultCerebellum,AdultCerebellum1,AdultCerebellum1_1,55Y,Adult,Microwell-seq,Female,HCL,AdultCerebellum,AdultCerebellum_55Y,1,Oligodendrocyte
4,AdultCerebellum_1.ATCTCTGCAGGATGCGGA,AdultCerebellum,AdultCerebellum1,AdultCerebellum1_1,55Y,Adult,Microwell-seq,Female,HCL,AdultCerebellum,AdultCerebellum_55Y,1,Oligodendrocyte
5,AdultCerebellum_1.CGAGTACTGTGTGCGAAT,AdultCerebellum,AdultCerebellum1,AdultCerebellum1_1,55Y,Adult,Microwell-seq,Female,HCL,AdultCerebellum,AdultCerebellum_55Y,1,Oligodendrocyte
6,AdultCerebellum_1.ACGAGCTAGAGACTCCAT,AdultCerebellum,AdultCerebellum1,AdultCerebellum1_1,55Y,Adult,Microwell-seq,Female,HCL,AdultCerebellum,AdultCerebellum_55Y,1,Oligodendrocyte


In [8]:
labels <- c('Biomaterial', 'Celltype')
datTraits <- allTraits[,labels]
rownames(datTraits) <- allTraits$Cell_id #<--CHECK
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,Biomaterial,Celltype
,<chr>,<chr>
AdultCerebellum_1.ACGTTGCCAGACGATCTT,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.CGAGTACGCACCGAGATC,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.GTATACGAATTAAATAAA,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.ATCTCTGCAGGATGCGGA,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.CGAGTACTGTGTGCGAAT,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.ACGAGCTAGAGACTCCAT,AdultCerebellum,Oligodendrocyte


In [9]:
table(dimnames(datExpr)[[1]]==datTraits$Biomaterial)


FALSE 
 2000 

In [10]:
y = datTraits$Biomaterial

In [11]:
meanExpressionBySample=apply( datExpr,1,mean, na.rm=T)
NumberMissingBySample=apply( is.na(data.frame(datExpr)),1, sum)

## 1

In [13]:
gsg = goodSamplesGenes(datExpr, verbose = 3);
gsg$allOK

 Flagging genes and samples with too many missing values...
  ..step 1


[1] TRUE

In [14]:
sampleTree = hclust(dist(datExpr), method = "average");
# Plot the sample tree: Open a graphic output window of size 12 by 9 inches
# The user should change the dimensions if the window is too large or too small.
sizeGrWindow(12,9)
svg("samplehier.svg")
par(cex = 0.6);
par(mar = c(0,4,2,0))
plot(sampleTree, main = "Sample clustering to detect outliers", sub="", xlab="", cex.lab = 1.5,
cex.axis = 1.5, cex.main = 2)
dev.off()

png 
  2

In [16]:
# Determine cluster under the line
clust = cutreeStatic(sampleTree, cutHeight = 15, minSize = 10)
table(clust)

clust
   0    1 
  64 1936 

In [17]:
# clust 1 contains the samples we want to keep.
keepSamples = (clust==1)
datExpr = datExpr[keepSamples, ]
nGenes = ncol(datExpr)
nSamples = nrow(datExpr)

genes <- dimnames(datExpr)[[2]]
samples <- dimnames(datExpr)[[1]]
datExpr <- as.data.frame(apply(datExpr,2, as.numeric)) #numeric
dimnames(datExpr)[[1]] <- samples

In [18]:
head(datExpr)

,A2M,AAMP,AASDHPPT,AB019441.29,ABAT,ABCA1,ABCA2,ABCA6,ABCA9,ABCB7,⋯,ZNF791,ZNF830,ZNF880,ZNFX1,ZNRD1,ZNRF1,ZSCAN18,ZSCAN30,ZSCAN9,ZWINT
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AdultCerebellum_1.ACGTTGCCAGACGATCTT,0,0,0.0000000,0,0,0,0.7597074,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AdultCerebellum_1.CGAGTACGCACCGAGATC,0,0,0.0000000,0,0,0,1.0694357,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AdultCerebellum_1.GTATACGAATTAAATAAA,0,0,0.0000000,0,0,0,1.2547720,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AdultCerebellum_1.ATCTCTGCAGGATGCGGA,0,0,0.0000000,0,0,0,0.7597074,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AdultCerebellum_1.ACGAGCTAGAGACTCCAT,0,0,0.7597074,0,0,0,0.0000000,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
AdultCerebellum_1.ATCAACAAAGTTTAAGGG,0,0,0.0000000,0,0,0,1.2547720,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [19]:
rownames(allTraits) <- allTraits$Cell_id
datTraits <- allTraits[keepSamples,labels]
rownames(datTraits) <- rownames(allTraits[keepSamples,labels])
datTraits <- datTraits[rownames(datExpr),]
head(datTraits)

,Biomaterial,Celltype
,<chr>,<chr>
AdultCerebellum_1.ACGTTGCCAGACGATCTT,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.CGAGTACGCACCGAGATC,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.GTATACGAATTAAATAAA,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.ATCTCTGCAGGATGCGGA,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.ACGAGCTAGAGACTCCAT,AdultCerebellum,Oligodendrocyte
AdultCerebellum_1.ATCAACAAAGTTTAAGGG,AdultCerebellum,Oligodendrocyte


In [20]:
collectGarbage();

In [21]:
# Re-cluster samples
sampleTree2 = hclust(dist(datExpr), method = "average")
fontsize=.5
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("samplehiercut.pdf")
plotDendroAndColors(sampleTree2, traitColors,
    groupLabels = labels,
    main = "Sample dendrogram and trait heatmap",
    dendroLabels=FALSE,
    rowText=datTraits,
    rowTextIgnore=labels[[2]]
    )
dev.off()

pdf 
  2

## 2

In [22]:
# Choose a set of soft-thresholding powers
powers = c(c(1:10), seq(from = 12, to=50, by=2))
# Call the network topology analysis function
sft = pickSoftThreshold(datExpr, powerVector = powers, verbose = 5)
# Plot the results:
sizeGrWindow(9, 5)
pdf("scaletopology.pdf")
par(mfrow = c(1,2));
cex1 = 0.9;
# Scale-free topology fit index as a function of the soft-thresholding power
plot(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
xlab="Soft Threshold (power)",ylab="Scale Free Topology Model Fit,signed R^2",type="n",
main = paste("Scale independence"));
text(sft$fitIndices[,1], -sign(sft$fitIndices[,3])*sft$fitIndices[,2],
labels=powers,cex=cex1,col="red");
# this line corresponds to using an R^2 cut-off of h
abline(h=0.90,col="red")
# Mean connectivity as a function of the soft-thresholding power
plot(sft$fitIndices[,1], sft$fitIndices[,5],
xlab="Soft Threshold (power)",ylab="Mean Connectivity", type="n",
main = paste("Mean connectivity"))
text(sft$fitIndices[,1], sft$fitIndices[,5], labels=powers, cex=cex1,col="red")
dev.off()

pickSoftThreshold: will use block size 3001.
 pickSoftThreshold: calculating connectivity for given powers...
   ..working on genes 1 through 3001 of 3001
   Power SFT.R.sq  slope truncated.R.sq  mean.k. median.k.   max.k.
1      1   0.0471 -0.800         0.8680 6.62e+01  6.20e+01 1.67e+02
2      2   0.9860 -3.010         0.9970 2.93e+00  2.21e+00 2.15e+01
3      3   0.9600 -2.170         0.9690 2.67e-01  1.26e-01 5.74e+00
4      4   0.3980 -2.450         0.3490 4.70e-02  1.00e-02 2.45e+00
5      5   0.9470 -1.550         0.9540 1.34e-02  9.86e-04 1.28e+00
6      6   0.4100 -1.920         0.3540 5.10e-03  1.11e-04 7.27e-01
7      7   0.4120 -1.790         0.3620 2.32e-03  1.41e-05 4.33e-01
8      8   0.3640 -2.030         0.3400 1.18e-03  1.88e-06 2.66e-01
9      9   0.3660 -1.930         0.3480 6.47e-04  2.65e-07 1.67e-01
10    10   0.3670 -2.120         0.3430 3.75e-04  3.82e-08 1.07e-01
11    12   0.3520 -1.720         0.3230 1.40e-04  8.86e-10 4.47e-02
12    14   0.3840 -1.330     

png 
  2

In [32]:
net = blockwiseModules(datExpr, power = 20,
TOMType = "unsigned", minModuleSize = 5,
reassignThreshold = 0, mergeCutHeight = 0.999999,
numericLabels = TRUE, pamRespectsDendro = FALSE,
saveTOMs = TRUE,
saveTOMFileBase = "wgcna",
verbose = 3)

 Calculating module eigengenes block-wise from all genes


   Flagging genes and samples with too many missing values...
    ..step 1
 ..Working on block 1 .
    TOM calculation: adjacency..
    ..will use 12 parallel threads.
     Fraction of slow calculations: 0.000000
    ..connectivity..
    ..matrix multiplication (system BLAS)..
    ..normalization..
    ..done.
   ..saving TOM for block 1 into file wgcna-block.1.RData
 ....clustering..
 ....detecting modules..
 No modules detected in block 1
 ..merging modules that are too close..
     mergeCloseModules: Merging modules whose distance is less than 0.999999


Warning message in blockwiseModules(datExpr, power = 20, TOMType = "unsigned", minModuleSize = 5, :
“blockwiseModules: mergeCloseModules failed with the following error message:
     Error in mergeCloseModules(datExpr, colors[gsg$goodGenes], cutHeight = mergeCutHeight,  : 
  Error in moduleEigengenes(expr = exprData[[set]]$data, colors = setColors,  : 
  Color levels are empty. Possible reason: the only color is grey and grey module is excluded from the calculation.

 
--> returning unmerged colors.
”


In [33]:
# open a graphics window
sizeGrWindow(12, 9)
pdf("netdendograms.pdf")
# Convert labels to colors for plotting
mergedColors = labels2colors(net$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(net$dendrograms[[1]], mergedColors[net$blockGenes[[1]]],
"Module colors",
dendroLabels = FALSE, hang = 0.03,
addGuide = TRUE, guideHang = 0.05)
dev.off()

png 
  2

In [34]:
moduleLabels = net$colors
moduleColors = labels2colors(net$colors)
MEs = net$MEs;
geneTree = net$dendrograms[[1]];

## 3

In [35]:
datTraits$Biomaterial <- match(datTraits$Biomaterial, unique(datTraits$Biomaterial))
datTraits$Celltype <- match(datTraits$Celltype, unique(datTraits$Celltype))

In [36]:
# Define numbers of genes and samples
nGenes = ncol(datExpr);
nSamples = nrow(datExpr);
# Recalculate MEs with color labels
MEs0 = moduleEigengenes(datExpr, moduleColors)$eigengenes
MEs = orderMEs(MEs0)
moduleTraitCor = cor(MEs, datTraits, use = "p");
moduleTraitPvalue = corPvalueStudent(moduleTraitCor, nSamples);

In [37]:
pdf("moduleTrait.pdf", width=9, height=18)
# Will display correlations and their p-values
textMatrix = paste(signif(moduleTraitCor, 2), "\n(",
signif(moduleTraitPvalue, 1), ")", sep = "");
dim(textMatrix) = dim(moduleTraitCor)
par(mar = c(6, 8.5, 3, 3));
# Display the correlation values within a heatmap plot
labeledHeatmap(Matrix = moduleTraitCor,
xLabels = labels,
yLabels = names(MEs),
ySymbols = names(MEs),
colorLabels = FALSE,
colors = blueWhiteRed(50),
textMatrix = textMatrix,
setStdMargins = FALSE,
cex.text = 0.5,
zlim = c(-1,1),
main = paste("Module-trait relationships"))
dev.off()

pdf 
  2

In [38]:
# Define variable weight containing the weight column of datTrait
weight = as.data.frame(rownames(datTraits));
names(weight) = "weight"
# names (colors) of the modules
modNames = substring(names(MEs), 3)
geneModuleMembership = as.data.frame(cor(datExpr, MEs, use = "p"));
MMPvalue = as.data.frame(corPvalueStudent(as.matrix(geneModuleMembership), nSamples));
names(geneModuleMembership) = paste("MM", modNames, sep="");
names(MMPvalue) = paste("p.MM", modNames, sep="");
geneTraitSignificance = as.data.frame(cor(datExpr, weight, use = "p"));
GSPvalue = as.data.frame(corPvalueStudent(as.matrix(geneTraitSignificance), nSamples));
names(geneTraitSignificance) = paste("GS.", names(weight), sep="");
names(GSPvalue) = paste("p.GS.", names(weight), sep="");

Warning message in storage.mode(y) <- "double":
“NAs introduced by coercion”


In [40]:
module = "gray"
column = match(module, modNames);
moduleGenes = moduleColors==module;
sizeGrWindow(7, 7);
pdf("modulemembership.pdf")
par(mfrow = c(1,1));
verboseScatterplot(abs(geneModuleMembership[moduleGenes, column]),
    abs(geneTraitSignificance[moduleGenes, 1]),
    xlab = paste("Module Membership in", module, "module"),
    ylab = "Gene significance for primary_site",
    main = paste("Module membership vs. gene significance\n"),
    cex.main = 1.2, cex.lab = 1.2, cex.axis = 1.2, col = module)
dev.off()

Warning message in sqrt(nSamples - 2):
“NaNs produced”
Warning message in min(x):
“no non-missing arguments to min; returning Inf”
Warning message in max(x):
“no non-missing arguments to max; returning -Inf”


ERROR: Error in plot.window(...): need finite 'ylim' values


In [41]:
# Create the starting data frame
geneInfo0 = data.frame(
moduleColor = moduleColors,
geneTraitSignificance,
GSPvalue)
# Order modules by their significance for weight
modOrder = order(-abs(cor(MEs, weight, use = "p")));
# Add module membership information in the chosen order
for (mod in 1:ncol(geneModuleMembership))
{
oldNames = names(geneInfo0)
geneInfo0 = data.frame(geneInfo0, geneModuleMembership[, modOrder[mod]],
MMPvalue[, modOrder[mod]]);
names(geneInfo0) = c(oldNames, paste("MM.", modNames[modOrder[mod]], sep=""),
paste("p.MM.", modNames[modOrder[mod]], sep=""))
}
# Order the genes in the geneInfo variable first by module color, then by geneTraitSignificance
geneOrder = order(geneInfo0$moduleColor, -abs(geneInfo0$GS.weight));
geneInfo = geneInfo0[geneOrder, ]
write.csv(geneInfo, file = "geneInfo.csv")

Warning message in storage.mode(y) <- "double":
“NAs introduced by coercion”


## 5 

In [42]:
# Re-cluster samples
sampleTree3 = hclust(dist(MEs0), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits);
# Plot the sample dendrogram and the colors underneath.
pdf("nethier.pdf", width=10)
# Set the plot margin: bottom, left, top & right
par(mar = c(5, 4, 1.4, 0.2) + 0.1,
    xpd = NA) # allow content to go into outer margin 
plotDendroAndColors(sampleTree3, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE,
#rowText=allTraits[keepSamples,labels],
#rowTextIgnore=labels[[2]],
addGuide=TRUE)
dev.off()

pdf 
  2

In [43]:
rownames(allTraits)=allTraits$Cell_id
unique(allTraits[rownames(datTraits),'Biomaterial'])

[1] "AdultCerebellum" "AdultHeart"      "AdultLiver"      "AdultLung"

In [44]:
svg("nethier_legend.svg", width=10)
add_legend("topleft", legend = unique(allTraits[rownames(datTraits),'Biomaterial']), pch = 1, pt.cex = 1, cex = 1.5, bty = 'n', ncol=2,
       inset = c(0.1, 0.), # place outside
       title = "Biomaterial", 
       col = traitColors)
dev.off()

Warning message in par(fig = c(0, 1, 0, 1), oma = c(0, 0, 0, 0), mar = c(0, 0, 0, :
“calling par(new=TRUE) with no plot”


pdf 
  2

In [45]:
collectGarbage()

In [47]:
clust = cutreeStatic(sampleTree3, cutHeight = 0.005, minSize = 10)
table(clust)

clust
  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15 
 12 509 345 291 211 121  87  81  59  52  48  40  29  22  18  11 

In [48]:
# Re-cluster samples after cut
sampleTree4 = hclust(dist(MEs0[(clust==1),]), method = "average")
# Convert traits to a color representation: white means low, red means high, grey means missing entry
traitColors = labels2colors(datTraits[(clust==1),]);
# Plot the sample dendrogram and the colors underneath.
svg("nethiercut.svg")
plotDendroAndColors(sampleTree4, traitColors,
groupLabels = labels,
main = "Network dendrogram and trait heatmap",
dendroLabels=FALSE)
dev.off()

pdf 
  2

In [49]:
# Calculate topological overlap anew: this could be done more efficiently by saving the TOM
# calculated during module detection, but let us do it again here.
dissTOM = 1-TOMsimilarityFromExpr(datExpr, power = 6);
# Transform dissTOM with a power to make moderately strong connections more visible in the heatmap
plotTOM = dissTOM^7;
# Set diagonal to NA for a nicer plot
diag(plotTOM) = NA;
# Call the plot function
sizeGrWindow(9,9)
svg("networkmap.svg")
TOMplot(plotTOM, geneTree, moduleColors, main = "Network heatmap plot, all genes")
dev.off()

TOM calculation: adjacency..
..will use 12 parallel threads.
 Fraction of slow calculations: 0.000000
..connectivity..
..matrix multiplication (system BLAS)..
..normalization..
..done.


png 
  2

In [50]:
#selct genes to be faster
nSelect = nGenes*0.7
# For reproducibility, we set the random seed
set.seed(10);
select = sample(nGenes, size = nSelect);
selectTOM = dissTOM[select, select];
# There’s no simple way of restricting a clustering tree to a subset of genes, so we must re-cluster.
selectTree = hclust(as.dist(selectTOM), method = "average")
selectColors = moduleColors[select];
# Open a graphical window
sizeGrWindow(9,9)
svg("networkmap.svg")
# Taking the dissimilarity to a power, say 10, makes the plot more informative by effectively changing
# the color palette; setting the diagonal to NA also improves the clarity of the plot
plotDiss = selectTOM^7;
diag(plotDiss) = NA;
TOMplot(plotDiss, selectTree, selectColors, main = "Network heatmap plot, selected genes")
dev.off()

png 
  2

In [51]:
# Recalculate module eigengenes
MEs = moduleEigengenes(datExpr, moduleColors)$eigengenes
# Isolate weight from the clinical traits
weight = as.data.frame(datTraits$Biomaterial);
names(weight) = "Biomaterial"
# Add the weight to existing module eigengenes
MET = orderMEs(cbind(MEs, weight))
# Plot the relationships among the eigengenes and the trait
sizeGrWindow(5,7.5);
svg("eigenHeat.svg")
par(cex = 0.9)
plotEigengeneNetworks(MET, "", marDendro = c(0,4,1,2), marHeatmap = c(3,4,1,2), cex.lab = 0.8, xLabelsAngle = 90)
dev.off()

ERROR: Error in multiME[[set]]$data[substring(labels, 3) != greyLabel, substring(labels, : incorrect number of dimensions


In [52]:
# Plot the dendrogram
sizeGrWindow(6,6);
svg("eigendendogram.svg")
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene dendrogram", marDendro = c(0,4,2,0),
plotHeatmaps = FALSE)
# Plot the heatmap matrix (note: this plot will overwrite the dendrogram plot)
par(cex = 1.0)
plotEigengeneNetworks(MET, "Eigengene adjacency heatmap", marHeatmap = c(3,4,2,2),
plotDendrograms = FALSE, xLabelsAngle = 90)
dev.off()

ERROR: Error in multiME[[set]]$data[substring(labels, 3) != greyLabel, substring(labels, : incorrect number of dimensions


Values of *k* must be read from *clustersizes.txt file*

In [61]:
write.csv(cutree(sampleTree3, k=4), "wgcna/wgcna_level_0_labels.csv")

In [56]:
write.csv(geneModuleMembership, "wgcna/wgcna_level_0_word-dist.csv")

In [57]:
write.csv(MEs, "wgcna/wgcna_level_0_topic-dist.csv")